In [1]:
import pandas as pd
import timeit
import numpy as np
from datetime import datetime
from datetime import timedelta
from math import sqrt
from sklearn.metrics import mean_squared_error
import warnings
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima_model import ARIMA
from math import sqrt

dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')
dataFrame = pd.read_csv(r"C:\Macros\VENDA-GAM-343.CSV", sep=";", parse_dates=['dt_venda'], index_col='dt_venda', date_parser=dateparse)

In [9]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto


def train_test_split(dfr):
    train_size = int(len(dfr) * 0.70)
    train = dfr[0: train_size]
    test = dfr[train_size:]
    return train, test

def suav_exp_simp():
    y_hat_avg = pd.DataFrame(test.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6, optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(test))
    rmse_ses = sqrt(mean_squared_error(test, y_hat_avg.SES))
    

dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]

In [7]:
inicio = timeit.default_timer()

arrayMercadoria = dataFrame['cd_mercadoria'].unique()
arrayEmpresa = dataFrame['cd_empresa'].unique()
arrayDeDataFrame = []

for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) &
                       (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'], ascending=True)
        if len(df) > 0:
            idx = pd.date_range(min(df.index), dataAtual, freq='MS')
            df.index = pd.DatetimeIndex(df.index)
            df = df.reindex(idx, fill_value=0)
            arrayDeDataFrame.append(df)
        arrayDeDataFrame[len(arrayDeDataFrame) - 1].fillna(0, inplace=True)

fim = timeit.default_timer()

print(f'Tempo de execução: {(fim - inicio) / 60} minutos')

Tempo de execução: 0.11370912498333326 minutos


In [12]:
for i in enumerate(arrayDeDataFrame):
    train_test_split(arrayDeDataFrame[i]['qt_vendida'])
    suav_exp_simp()
    
    

TypeError: list indices must be integers or slices, not tuple